## Google Colab Rust Setup

The following cell is used to set up a Rust environment on Colab. Don't execute it locally!

Many thanks to [`mateusvmv`](https://github.com/mateusvmv) for this hack in [`gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3`](https://gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3?permalink_comment_id=4715636#gistcomment-4715636).

In [ ]:
# This script sets up and spins up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Install Required Dependencies

In [2]:
:dep plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
// or
// :dep plotters = { git = "https://github.com/plotters-rs/plotters"}"}

In [3]:
:dep polars = {version = "0.35.0", features = ["describe", "lazy", "ndarray"]}
// or
// :dep polars = { git = "https://github.com/pola-rs/polars"}"}

In [4]:
:dep ndarray = {version = "0.15.6"}
// or
// :dep ndarray = { git = "https://github.com/rust-ndarray/ndarray"}

In [7]:
:dep smartcore = {version = "0.3.2"}
// or
// :dep smartcore = { git = "https://github.com/smartcorelib/smartcore"}"}

## Show Installed Dependencies

In [10]:
:show_deps

ndarray = {version = "0.15.6"}
plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
polars = {version = "0.35.0", features = ["describe", "lazy", "ndarray"]}
smartcore = {version = "0.3.2"}


## Import Modules

In [11]:
use polars::prelude::*;
use polars::frame::DataFrame;
use std::path::Path;
use ndarray::{ArrayBase, DataMut, Dimension, concatenate, Axis};
use plotters::prelude::*;
use smartcore::linalg::basic::matrix::DenseMatrix;
use ndarray::prelude::*;

### Display Documentation

In [15]:
:doc DataFrame

polars_core::frame

pub struct DataFrame {
    pub(crate) columns: Vec<Series, Global>,
}


A contiguous growable collection of Series that have the same length.

Use declarationsAll the common tools can be found in crate::prelude (or in polars::prelude).

use polars_core::prelude::*; // if the crate polars-core is used directly
// use polars::prelude::*;      if the crate polars is used

InitializationDefaultA DataFrame can be initialized empty:

# use polars_core::prelude::*;
let df = DataFrame::default();
assert!(df.is_empty());

Wrapping a Vec<Series>A DataFrame is built upon a Vec<Series> where the Series have the same length.

# use polars_core::prelude::*;
let s1 = Series::new("Fruit", &["Apple", "Apple", "Pear"]);
let s2 = Series::new("Color", &["Red", "Yellow", "Green"]);

let df: PolarsResult<DataFrame> = DataFrame::new(vec![s1, s2]);

Using a macroThe df macro is a convenient method:

# use polars_core::prelude::*;
let df: PolarsResult<DataFrame> = df!("Fruit" => &["Apple", "Apple", "Pear"],
                                      "Color" => &["Red", "Yellow", "Green"]);

Using a CSV fileSee the polars_io::csv::CsvReader.

IndexingBy a numberThe Index<usize> is implemented for the DataFrame.

# use polars_core::prelude::*;
let df = df!("Fruit" => &["Apple", "Apple", "Pear"],
             "Color" => &["Red", "Yellow", "Green"])?;

assert_eq!(df[0], Series::new("Fruit", &["Apple", "Apple", "Pear"]));
assert_eq!(df[1], Series::new("Color", &["Red", "Yellow", "Green"]));
# Ok::<(), PolarsError>(())

By a Series name# use polars_core::prelude::*;
let df = df!("Fruit" => &["Apple", "Apple", "Pear"],
             "Color" => &["Red", "Yellow", "Green"])?;

assert_eq!(df["Fruit"], Series::new("Fruit", &["Apple", "Apple", "Pear"]));
assert_eq!(df["Color"], Series::new("Color", &["Red", "Yellow", "Green"]));
# Ok::<(), PolarsError>(())


## Load Dataset

In [13]:
fn read_data_frame_from_csv(
    csv_file_path: &Path,
) -> DataFrame {
    CsvReader::from_path(csv_file_path)
        .expect("Cannot open file.")
        .has_header(true)
        .finish()
        .unwrap()
}

let iris_file_path: &Path = Path::new("dataset/Iris.csv");
let iris_df: DataFrame = read_data_frame_from_csv(iris_file_path);
iris_df

shape: (150, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬────────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species        │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---            │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str            │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪════════════════╡
│ 1   ┆ 5.1           ┆ 3.5          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa    │
│ 2   ┆ 4.9           ┆ 3.0          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa    │
│ 3   ┆ 4.7           ┆ 3.2          ┆ 1.3           ┆ 0.2          ┆ Iris-setosa    │
│ 4   ┆ 4.6           ┆ 3.1          ┆ 1.5           ┆ 0.2          ┆ Iris-setosa    │
│ …   ┆ …             ┆ …            ┆ …             ┆ …            ┆ …              │
│ 147 ┆ 6.3           ┆ 2.5          ┆ 5.0           ┆ 1.9          ┆ Iris-virginica │
│ 148 ┆ 6.5           ┆ 3.0

In [14]:
:type iris_df

iris_df: DataFrame

## About The Dataset

### Shape

In [16]:
iris_df.shape()

(150, 6)

### Head

In [17]:
iris_df.head(Some(5))

shape: (5, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬─────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species     │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---         │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str         │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪═════════════╡
│ 1   ┆ 5.1           ┆ 3.5          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa │
│ 2   ┆ 4.9           ┆ 3.0          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa │
│ 3   ┆ 4.7           ┆ 3.2          ┆ 1.3           ┆ 0.2          ┆ Iris-setosa │
│ 4   ┆ 4.6           ┆ 3.1          ┆ 1.5           ┆ 0.2          ┆ Iris-setosa │
│ 5   ┆ 5.0           ┆ 3.6          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa │
└─────┴───────────────┴──────────────┴───────────────┴──────────────┴─────────────┘

### Tail

In [18]:
iris_df.tail(Some(5))

shape: (5, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬────────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species        │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---            │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str            │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪════════════════╡
│ 146 ┆ 6.7           ┆ 3.0          ┆ 5.2           ┆ 2.3          ┆ Iris-virginica │
│ 147 ┆ 6.3           ┆ 2.5          ┆ 5.0           ┆ 1.9          ┆ Iris-virginica │
│ 148 ┆ 6.5           ┆ 3.0          ┆ 5.2           ┆ 2.0          ┆ Iris-virginica │
│ 149 ┆ 6.2           ┆ 3.4          ┆ 5.4           ┆ 2.3          ┆ Iris-virginica │
│ 150 ┆ 5.9           ┆ 3.0          ┆ 5.1           ┆ 1.8          ┆ Iris-virginica │
└─────┴───────────────┴──────────────┴───────────────┴──────────────┴────────────────┘

### Describe

In [19]:
iris_df.describe(None)?

shape: (9, 7)
┌────────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┐
│ describe   ┆ Id        ┆ SepalLengthC ┆ SepalWidthCm ┆ PetalLengthC ┆ PetalWidthCm ┆ Species     │
│ ---        ┆ ---       ┆ m            ┆ ---          ┆ m            ┆ ---          ┆ ---         │
│ str        ┆ f64       ┆ ---          ┆ f64          ┆ ---          ┆ f64          ┆ str         │
│            ┆           ┆ f64          ┆              ┆ f64          ┆              ┆             │
╞════════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════════╡
│ count      ┆ 150.0     ┆ 150.0        ┆ 150.0        ┆ 150.0        ┆ 150.0        ┆ 150         │
│ null_count ┆ 0.0       ┆ 0.0          ┆ 0.0          ┆ 0.0          ┆ 0.0          ┆ 0           │
│ mean       ┆ 75.5      ┆ 5.843333     ┆ 3.054        ┆ 3.758667     ┆ 1.198667     ┆ null        │
│ std        ┆ 43.445368 ┆ 0.828066     ┆ 0.433594     ┆ 1.76442      ┆ 0.763

### Columns

In [20]:
let column_names = iris_df.get_column_names(); 

{
    column_names
}

["Id", "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Species"]

### Drop Species Column

In [21]:
let numeric_iris_df: DataFrame = iris_df.drop("Species")?;

### Mean

In [22]:
numeric_iris_df.mean()

shape: (1, 5)
┌──────┬───────────────┬──────────────┬───────────────┬──────────────┐
│ Id   ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm │
│ ---  ┆ ---           ┆ ---          ┆ ---           ┆ ---          │
│ f64  ┆ f64           ┆ f64          ┆ f64           ┆ f64          │
╞══════╪═══════════════╪══════════════╪═══════════════╪══════════════╡
│ 75.5 ┆ 5.843333      ┆ 3.054        ┆ 3.758667      ┆ 1.198667     │
└──────┴───────────────┴──────────────┴───────────────┴──────────────┘

### Max

In [23]:
numeric_iris_df.max()

shape: (1, 5)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╡
│ 150 ┆ 7.9           ┆ 4.4          ┆ 6.9           ┆ 2.5          │
└─────┴───────────────┴──────────────┴───────────────┴──────────────┘

### Comvert To ndarray

In [26]:
let numeric_iris_ndarray: ArrayBase<_, _> = numeric_iris_df.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap();
numeric_iris_ndarray

[[1.0, 5.1, 3.5, 1.4, 0.2],
 [2.0, 4.9, 3.0, 1.4, 0.2],
 [3.0, 4.7, 3.2, 1.3, 0.2],
 [4.0, 4.6, 3.1, 1.5, 0.2],
 [5.0, 5.0, 3.6, 1.4, 0.2],
 ...,
 [146.0, 6.7, 3.0, 5.2, 2.3],
 [147.0, 6.3, 2.5, 5.0, 1.9],
 [148.0, 6.5, 3.0, 5.2, 2.0],
 [149.0, 6.2, 3.4, 5.4, 2.3],
 [150.0, 5.9, 3.0, 5.1, 1.8]], shape=[150, 5], strides=[1, 150], layout=Ff (0xa), const ndim=2

<hr />

## ndarray

### First Element

In [27]:
numeric_iris_ndarray[[0, 0]]

1.0

### First Row

In [28]:
numeric_iris_ndarray.slice(s![0, ..])

[1.0, 5.1, 3.5, 1.4, 0.2], shape=[5], strides=[150], layout=Custom (0x0), const ndim=1

### Shape (rows, cols)

In [29]:
numeric_iris_ndarray.dim()

(150, 5)

### Last Row

In [30]:
let numeric_iris_ndarray_shape = numeric_iris_ndarray.dim(); // (rows, cols)
// numeric_iris_ndarray.slice(s![numeric_iris_ndarray_shape.1 -1, ..])
// or
numeric_iris_ndarray.slice(s![-1, ..])

[150.0, 5.9, 3.0, 5.1, 1.8], shape=[5], strides=[150], layout=Custom (0x0), const ndim=1

### First Five Row

In [31]:
numeric_iris_ndarray.slice(s![0..5, ..])

[[1.0, 5.1, 3.5, 1.4, 0.2],
 [2.0, 4.9, 3.0, 1.4, 0.2],
 [3.0, 4.7, 3.2, 1.3, 0.2],
 [4.0, 4.6, 3.1, 1.5, 0.2],
 [5.0, 5.0, 3.6, 1.4, 0.2]], shape=[5, 5], strides=[1, 150], layout=f (0x8), const ndim=2

### Last Five Row

In [32]:
numeric_iris_ndarray.slice(s![-5.., ..])

[[146.0, 6.7, 3.0, 5.2, 2.3],
 [147.0, 6.3, 2.5, 5.0, 1.9],
 [148.0, 6.5, 3.0, 5.2, 2.0],
 [149.0, 6.2, 3.4, 5.4, 2.3],
 [150.0, 5.9, 3.0, 5.1, 1.8]], shape=[5, 5], strides=[1, 150], layout=f (0x8), const ndim=2

### Check for zero elements

In [33]:
numeric_iris_ndarray.is_empty()

false

<hr />

## Mathematics

### Sum

In [34]:
numeric_iris_ndarray.sum()

13403.199999999997

### Sum Along Axis

In [35]:
numeric_iris_ndarray.sum_axis(Axis(0))

[11325.0, 876.5000000000001, 458.1, 563.8000000000002, 179.80000000000004], shape=[5], strides=[1], layout=CFcf (0xf), const ndim=1

### Mean

In [36]:
numeric_iris_ndarray.mean().unwrap()

17.87093333333333

### Transpose

In [37]:
numeric_iris_ndarray.t()

[[1.0, 2.0, 3.0, 4.0, 5.0, ..., 146.0, 147.0, 148.0, 149.0, 150.0],
 [5.1, 4.9, 4.7, 4.6, 5.0, ..., 6.7, 6.3, 6.5, 6.2, 5.9],
 [3.5, 3.0, 3.2, 3.1, 3.6, ..., 3.0, 2.5, 3.0, 3.4, 3.0],
 [1.4, 1.4, 1.3, 1.5, 1.4, ..., 5.2, 5.0, 5.2, 5.4, 5.1],
 [0.2, 0.2, 0.2, 0.2, 0.2, ..., 2.3, 1.9, 2.0, 2.3, 1.8]], shape=[5, 150], strides=[150, 1], layout=Cc (0x5), const ndim=2

### 2-D matrix multiply

In [38]:
numeric_iris_ndarray.dot(&numeric_iris_ndarray.t())

[[41.26, 39.489999999999995, 40.03, 40.449999999999996, 45.1, ..., 198.41, 195.26, 199.33, 200.54000000000002, 198.09],
 [39.489999999999995, 39.010000000000005, 40.49, 41.98, 47.3, ..., 341.56999999999994, 339.75, 344.53, 346.59999999999997, 345.41],
 [40.03, 40.49, 43.06, 45.53, 51.88, ..., 486.31, 485.49, 491.31, 494.49999999999994, 494.32000000000005],
 [40.449999999999996, 41.98, 45.53, 49.059999999999995, 56.300000000000004, ..., 632.38, 632.61, 639.3999999999999, 643.62, 644.4499999999999],
 [45.1, 47.3, 51.88, 56.300000000000004, 64.96000000000001, ..., 782.04, 782.88, 790.9799999999999, 796.26, 797.8],
 ...,
 [198.41, 341.56999999999994, 486.31, 632.38, 782.04, ..., 21402.22, 21542.079999999998, 21692.19, 21839.110000000004, 21979.19],
 [195.26, 339.75, 485.49, 632.61, 782.88, ..., 21542.079999999998, 21683.55, 21834.25, 21981.93, 22123.589999999997],
 [199.33, 344.53, 491.31, 639.3999999999999, 790.9799999999999, ..., 21692.19, 21834.25, 21986.29, 22135.18, 22277.469999999998

<hr />

## Arithmetics

In [39]:
&numeric_iris_ndarray + 1.0

[[2.0, 6.1, 4.5, 2.4, 1.2],
 [3.0, 5.9, 4.0, 2.4, 1.2],
 [4.0, 5.7, 4.2, 2.3, 1.2],
 [5.0, 5.6, 4.1, 2.5, 1.2],
 [6.0, 6.0, 4.6, 2.4, 1.2],
 ...,
 [147.0, 7.7, 4.0, 6.2, 3.3],
 [148.0, 7.3, 3.5, 6.0, 2.9],
 [149.0, 7.5, 4.0, 6.2, 3.0],
 [150.0, 7.2, 4.4, 6.4, 3.3],
 [151.0, 6.9, 4.0, 6.1, 2.8]], shape=[150, 5], strides=[1, 150], layout=Ff (0xa), const ndim=2

<hr />

## Sepal Length Column

In [40]:
numeric_iris_ndarray.column(1)

[5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 4.6, 5.0, 4.4, 4.9, 5.4, 4.8, 4.8, 4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1, 5.4, 5.1, 4.6, 5.1, 4.8, 5.0, 5.0, 5.2, 5.2, 4.7, 4.8, 5.4, 5.2, 5.5, 4.9, 5.0, 5.5, 4.9, 4.4, 5.1, 5.0, 4.5, 4.4, 5.0, 5.1, 4.8, 5.1, 4.6, 5.3, 5.0, 7.0, 6.4, 6.9, 5.5, 6.5, 5.7, 6.3, 4.9, 6.6, 5.2, 5.0, 5.9, 6.0, 6.1, 5.6, 6.7, 5.6, 5.8, 6.2, 5.6, 5.9, 6.1, 6.3, 6.1, 6.4, 6.6, 6.8, 6.7, 6.0, 5.7, 5.5, 5.5, 5.8, 6.0, 5.4, 6.0, 6.7, 6.3, 5.6, 5.5, 5.5, 6.1, 5.8, 5.0, 5.6, 5.7, 5.7, 6.2, 5.1, 5.7, 6.3, 5.8, 7.1, 6.3, 6.5, 7.6, 4.9, 7.3, 6.7, 7.2, 6.5, 6.4, 6.8, 5.7, 5.8, 6.4, 6.5, 7.7, 7.7, 6.0, 6.9, 5.6, 7.7, 6.3, 6.7, 7.2, 6.2, 6.1, 6.4, 7.2, 7.4, 7.9, 6.4, 6.3, 6.1, 7.7, 6.3, 6.4, 6.0, 6.9, 6.7, 6.9, 5.8, 6.8, 6.7, 6.7, 6.3, 6.5, 6.2, 5.9], shape=[150], strides=[1], layout=CFcf (0xf), const ndim=1

Many thanks to [Brendan A R Sechter](https://github.com/sgeos) for fixing this bug in [#17](https://github.com/wiseaidev/rust-data-analysis/issues/17).

In [41]:
{
  let axis = Axis(1); // dropped at end of block
  let array_data_a = numeric_iris_ndarray.column(1).insert_axis(axis); // dropped at end of block
  let array_data_b = numeric_iris_ndarray.column(2).insert_axis(axis); // dropped at end of block
  concatenate(axis, &[array_data_a, array_data_b])? // returned by block
}

[[5.1, 3.5],
 [4.9, 3.0],
 [4.7, 3.2],
 [4.6, 3.1],
 [5.0, 3.6],
 [5.4, 3.9],
 [4.6, 3.4],
 [5.0, 3.4],
 [4.4, 2.9],
 [4.9, 3.1],
 [5.4, 3.7],
 [4.8, 3.4],
 [4.8, 3.0],
 [4.3, 3.0],
 [5.8, 4.0],
 [5.7, 4.4],
 [5.4, 3.9],
 [5.1, 3.5],
 [5.7, 3.8],
 [5.1, 3.8],
 [5.4, 3.4],
 [5.1, 3.7],
 [4.6, 3.6],
 [5.1, 3.3],
 [4.8, 3.4],
 [5.0, 3.0],
 [5.0, 3.4],
 [5.2, 3.5],
 [5.2, 3.4],
 [4.7, 3.2],
 [4.8, 3.1],
 [5.4, 3.4],
 [5.2, 4.1],
 [5.5, 4.2],
 [4.9, 3.1],
 [5.0, 3.2],
 [5.5, 3.5],
 [4.9, 3.1],
 [4.4, 3.0],
 [5.1, 3.4],
 [5.0, 3.5],
 [4.5, 2.3],
 [4.4, 3.2],
 [5.0, 3.5],
 [5.1, 3.8],
 [4.8, 3.0],
 [5.1, 3.8],
 [4.6, 3.2],
 [5.3, 3.7],
 [5.0, 3.3],
 [7.0, 3.2],
 [6.4, 3.2],
 [6.9, 3.1],
 [5.5, 2.3],
 [6.5, 2.8],
 [5.7, 2.8],
 [6.3, 3.3],
 [4.9, 2.4],
 [6.6, 2.9],
 [5.2, 2.7],
 [5.0, 2.0],
 [5.9, 3.0],
 [6.0, 2.2],
 [6.1, 2.9],
 [5.6, 2.9],
 [6.7, 3.1],
 [5.6, 3.0],
 [5.8, 2.7],
 [6.2, 2.2],
 [5.6, 2.5],
 [5.9, 3.2],
 [6.1, 2.8],
 [6.3, 2.5],
 [6.1, 2.8],
 [6.4, 2.9],
 [6.6, 3.0],
 [6.8, 2.8],

In [42]:
numeric_iris_df.get_column_names()

["Id", "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]

## Filter one label

In [43]:
let column_name = "Species";
let column_filter_value = "Iris-virginica";
let mask = iris_df.column(column_name)?.equal(column_filter_value)?;
let filtered_dataset = iris_df.filter(&mask);
filtered_dataset

Ok(shape: (50, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬────────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species        │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---            │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str            │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪════════════════╡
│ 101 ┆ 6.3           ┆ 3.3          ┆ 6.0           ┆ 2.5          ┆ Iris-virginica │
│ 102 ┆ 5.8           ┆ 2.7          ┆ 5.1           ┆ 1.9          ┆ Iris-virginica │
│ 103 ┆ 7.1           ┆ 3.0          ┆ 5.9           ┆ 2.1          ┆ Iris-virginica │
│ 104 ┆ 6.3           ┆ 2.9          ┆ 5.6           ┆ 1.8          ┆ Iris-virginica │
│ …   ┆ …             ┆ …            ┆ …             ┆ …            ┆ …              │
│ 147 ┆ 6.3           ┆ 2.5          ┆ 5.0           ┆ 1.9          ┆ Iris-virginica │
│ 148 ┆ 6.5           ┆ 3

<hr />

## Scatter Plot

### Sepal

In [44]:
let sepal_samples:Vec<(f64,f64)> = {
    let sepal_length_cm: DataFrame = iris_df.select(vec!["SepalLengthCm"]).unwrap();
    let mut sepal_length = sepal_length_cm.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec().into_iter();
    let sepal_width_cm: DataFrame = iris_df.select(vec!["SepalWidthCm"]).unwrap();
    let mut sepal_width = sepal_width_cm.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec().into_iter();
    sepal_width.zip(sepal_length).collect()
};

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Iris Dataset", ("Arial", 30).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 3f64..9f64)?;
    
    chart.configure_mesh()
        .x_desc("Sepal Length (cm)")
        .y_desc("Sepal Width (cm)")
        .draw()?;
    
    chart.draw_series(sepal_samples.iter().map(|(x, y)| Circle::new((*x,*y), 3, BLUE.filled())));

    Ok(())
}).style("width:60%")

Iris Dataset
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Sepal Width (cm)
 
 
Sepal Length (cm)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
9.0
 
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0

### Petal

In [45]:
let petal_samples: Vec<(f64,f64)> = {
    let petal_length_cm: DataFrame = iris_df.select(vec!["PetalLengthCm"]).unwrap();
    let mut petal_length = petal_length_cm.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec().into_iter();
    let petal_width_cm: DataFrame = iris_df.select(vec!["PetalWidthCm"]).unwrap();
    let mut petal_width = petal_width_cm.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec().into_iter();
    petal_width.zip(petal_length).collect()
};

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Iris Dataset", ("Arial", 30).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..3f64, 0f64..8f64)?;
    
    chart.configure_mesh()
        .x_desc("Petal Length (cm)")
        .y_desc("Petal Width (cm)")
        .draw()?;
    
    chart.draw_series(petal_samples.iter().map(|(x, y)| Circle::new((*x,*y), 3, GREEN.filled())));

    Ok(())
}).style("width:60%")

Iris Dataset
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Petal Width (cm)
 
 
Petal Length (cm)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0

## Histogram

In [46]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Histogram", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1u32..5u32, 0f64..1f64)?;

    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .y_labels(5)
        .y_label_formatter(&|y| format!("{}%", (*y * 100.0) as u32))
        .draw()?;

    let hist = Histogram::vertical(&chart)
        .style(RED.filled())
        .margin(0)
        .data(sepal_samples.iter().map(|(x,_)| (*x as u32, 0.01)));

    let _ = chart.draw_series(hist);
    
    Ok(())
}).style("width:100%")

Histogram
 
 
 
0%
 
 
 
50%
 
 
 
100%
 
 
 
 
1
 
 
 
2
 
 
 
3
 
 
 
4
 
 
 
5

## Combination of Histogram and Scatter

Many thanks to [Brendan A R Sechter](https://github.com/sgeos) for implementing this feature in [#17](https://github.com/wiseaidev/rust-data-analysis/issues/17).

In [47]:
evcxr_figure((640, 480), |root| {
    let root = root.titled("Scatter with Histogram Example", ("Arial", 20).into_font())?;
    
    let areas = root.split_by_breakpoints([560], [80]);

    let mut x_hist_ctx = ChartBuilder::on(&areas[0])
     .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1..5, 3..9)?;
    let mut y_hist_ctx = ChartBuilder::on(&areas[3])
        .x_label_area_size(40)
     .y_label_area_size(40)
        .build_cartesian_2d(1..5, 3..9)?;
    let mut scatter_ctx = ChartBuilder::on(&areas[2])
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 3f64..9f64)?;
    scatter_ctx.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    scatter_ctx.draw_series(sepal_samples.iter().map(|(x,y)| Circle::new((*x,*y), 3, BLUE.filled())))?;

    let x_hist = Histogram::vertical(&x_hist_ctx)
        .style(RED.filled())
        .margin(0)
        .data(sepal_samples.iter().map(|(x,_)| (*x as i32, 1)));
    x_hist_ctx.draw_series(x_hist)?;

    let y_hist = Histogram::horizontal(&y_hist_ctx)
        .style(GREEN.filled())
        .margin(0)
        .data(sepal_samples.iter().map(|(_,y)| (*y as i32, 1)));
    y_hist_ctx.draw_series(y_hist)?;

    Ok(())
}).style("width:60%")

Scatter with Histogram Example
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
9.0
 
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0

## ML Model

In [48]:
let features: DataFrame = iris_df.select(vec!["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]).unwrap();

In [49]:
let features: DataFrame = iris_df.select(vec!["SepalLengthCm"]).unwrap();
let array = features.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec();

In [50]:
array

[5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 4.6, 5.0, 4.4, 4.9, 5.4, 4.8, 4.8, 4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1, 5.4, 5.1, 4.6, 5.1, 4.8, 5.0, 5.0, 5.2, 5.2, 4.7, 4.8, 5.4, 5.2, 5.5, 4.9, 5.0, 5.5, 4.9, 4.4, 5.1, 5.0, 4.5, 4.4, 5.0, 5.1, 4.8, 5.1, 4.6, 5.3, 5.0, 7.0, 6.4, 6.9, 5.5, 6.5, 5.7, 6.3, 4.9, 6.6, 5.2, 5.0, 5.9, 6.0, 6.1, 5.6, 6.7, 5.6, 5.8, 6.2, 5.6, 5.9, 6.1, 6.3, 6.1, 6.4, 6.6, 6.8, 6.7, 6.0, 5.7, 5.5, 5.5, 5.8, 6.0, 5.4, 6.0, 6.7, 6.3, 5.6, 5.5, 5.5, 6.1, 5.8, 5.0, 5.6, 5.7, 5.7, 6.2, 5.1, 5.7, 6.3, 5.8, 7.1, 6.3, 6.5, 7.6, 4.9, 7.3, 6.7, 7.2, 6.5, 6.4, 6.8, 5.7, 5.8, 6.4, 6.5, 7.7, 7.7, 6.0, 6.9, 5.6, 7.7, 6.3, 6.7, 7.2, 6.2, 6.1, 6.4, 7.2, 7.4, 7.9, 6.4, 6.3, 6.1, 7.7, 6.3, 6.4, 6.0, 6.9, 6.7, 6.9, 5.8, 6.8, 6.7, 6.7, 6.3, 6.5, 6.2, 5.9]

In [51]:
let features: DataFrame = iris_df.select(vec!["SepalWidthCm"]).unwrap();
let array = features.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec();

In [52]:
array

[3.5, 3.0, 3.2, 3.1, 3.6, 3.9, 3.4, 3.4, 2.9, 3.1, 3.7, 3.4, 3.0, 3.0, 4.0, 4.4, 3.9, 3.5, 3.8, 3.8, 3.4, 3.7, 3.6, 3.3, 3.4, 3.0, 3.4, 3.5, 3.4, 3.2, 3.1, 3.4, 4.1, 4.2, 3.1, 3.2, 3.5, 3.1, 3.0, 3.4, 3.5, 2.3, 3.2, 3.5, 3.8, 3.0, 3.8, 3.2, 3.7, 3.3, 3.2, 3.2, 3.1, 2.3, 2.8, 2.8, 3.3, 2.4, 2.9, 2.7, 2.0, 3.0, 2.2, 2.9, 2.9, 3.1, 3.0, 2.7, 2.2, 2.5, 3.2, 2.8, 2.5, 2.8, 2.9, 3.0, 2.8, 3.0, 2.9, 2.6, 2.4, 2.4, 2.7, 2.7, 3.0, 3.4, 3.1, 2.3, 3.0, 2.5, 2.6, 3.0, 2.6, 2.3, 2.7, 3.0, 2.9, 2.9, 2.5, 2.8, 3.3, 2.7, 3.0, 2.9, 3.0, 3.0, 2.5, 2.9, 2.5, 3.6, 3.2, 2.7, 3.0, 2.5, 2.8, 3.2, 3.0, 3.8, 2.6, 2.2, 3.2, 2.8, 2.8, 2.7, 3.3, 3.2, 2.8, 3.0, 2.8, 3.0, 2.8, 3.8, 2.8, 2.8, 2.6, 3.0, 3.4, 3.1, 3.0, 3.1, 3.1, 3.1, 2.7, 3.2, 3.3, 3.0, 2.5, 3.0, 3.4, 3.0]

---
---